# TTP: advanced NN architectures

In [ ]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
#Importing libraries for preprocessing
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
#Importing libraries for model building
from tensorflow.keras import layers, regularizers
from tensorflow.keras.layers import LeakyReLU
#Importing libraries for model evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error
#Importing libraries for interpretability
import shap

# for reproducibility, the value is set for conventional reasons
SEED = 42
tf.keras.utils.set_random_seed(SEED)

In [ ]:
# load the data
data = pd.read_csv('dataset_d.csv', encoding='latin-1', sep=',') # request the dataset to the author
#data.head()

In [ ]:
# target column : "progression", regresion task
# relevant columns for the model
relevant_columns = [ 'age', 'sex', 'smoking', 'ps_at_diagnosis_ad', 'n#_mets_sites', 'lung_only_m1', 'pleural', 'pericard', 'lymph_nodes_only_m1', 'soft_tissue',
    'leptomingeal','skin','peritoneal','renal','pancreas', 'brain', 'liver', 'bone', 'adrenal', 'histology', 'hbbaselineio','leucotbaselineio',
    'neut_abs...143','linfo_abs...144','baso_abs...145', 'mono_abs...147', 'plaqtbaselineio', 'progression']

df = data[relevant_columns]
df.shape

In [ ]:
df = df.dropna(axis=0)
df.shape

In [ ]:
#to randomize the data
df = df.sample(frac=1, random_state=SEED)

var_int = ['ps_at_diagnosis_ad', 'n#_mets_sites', 'lung_only_m1', 'pleural', 'pericard', 'lymph_nodes_only_m1', 'soft_tissue',
    'leptomingeal','skin','peritoneal','renal','pancreas', 'brain', 'liver', 'bone', 'adrenal']
for i in var_int:
    df[i] = df[i].astype(int)
    
df['sex'] = df['sex'].str.lower()

# one-hot encoding
one_hot_data = pd.get_dummies(df, columns=['histology', 'sex', 'smoking'])

cols_to_convert = ['histology_adenocarcinoma','histology_nsclc', 'histology_squamous', 'sex_female', 'sex_male',
                   'smoking_current', 'smoking_former', 'smoking_non-smoker']

one_hot_data[cols_to_convert] = one_hot_data[cols_to_convert].astype(int)

In [ ]:
# split the data into features and target
X = one_hot_data[one_hot_data.columns.difference(['progression'])] 
y = df['progression']

In [ ]:
# split the data into training, validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

X_train.shape, X_val.shape, X_test.shape

In [ ]:
binary_features = ['lung_only_m1', 'pleural', 'pericard', 'lymph_nodes_only_m1', 'soft_tissue', 'leptomingeal','skin','peritoneal','renal',
                   'pancreas', 'brain', 'liver', 'bone', 'adrenal','histology_adenocarcinoma', 'histology_nsclc', 
                   'histology_squamous', 'sex_female', 'sex_male','smoking_current', 'smoking_former', 'smoking_non-smoker']
numeric_features = ['hbbaselineio','leucotbaselineio', 'neut_abs...143','linfo_abs...144','baso_abs...145', 'mono_abs...147',
                    'plaqtbaselineio', 'age', 'ps_at_diagnosis_ad', 'n#_mets_sites', ]


X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy() 
X_test_scaled = X_test.copy()
X_train_val_scaled = X_train_val.copy()

scaler = StandardScaler()

X_train_val_scaled[numeric_features] = scaler.fit_transform(X_train_val_scaled[numeric_features])
X_train_scaled[numeric_features] = scaler.fit_transform(X_train_scaled[numeric_features])
X_val_scaled[numeric_features] = scaler.transform(X_val_scaled[numeric_features])
X_test_scaled[numeric_features] = scaler.transform(X_test_scaled[numeric_features])

In [ ]:
def smape_f(y_true, y_pred):
    epsilon = tf.keras.backend.epsilon()
    denominator = tf.maximum(
        (tf.abs(y_true) + tf.abs(y_pred) + epsilon) / 2.0,
        epsilon
    )
    diff = tf.abs(y_true - y_pred)
    return 100 * tf.reduce_mean(diff / denominator)

## Funnel MLP

In [ ]:
# 1. Funnel MLP (Deep Funnel Network)
#    Input → 512 → BN → ReLU → DO(0.3)
#          → 256 → BN → ReLU → DO(0.3)
#          → 128 → BN → ReLU → DO(0.2)
#          →  64 → BN → ReLU → DO(0.2)
#          → Output

mlp_model = tf.keras.Sequential([
        layers.Dense(512, activation='relu', input_shape=(X.shape[1],)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),
        layers.Dense(1, activation='linear')  # Output layer for regression
    ])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()

In [ ]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)


## WideRes MLP

In [ ]:
# 2. WideRes MLP, Wide‑but‑Shallow Residual MLP
#    Input → 1024 → ReLU → DO(0.3) → add(input→dense(1024))
#          → 1024 → ReLU → DO(0.3)
#          → 1024 → ReLU → DO(0.3)
#          → Output

def build_model_residual_wide(input_dim):
    inp = layers.Input(shape=(input_dim,))
    x = layers.Dense(1024, activation='relu')(inp)
    x = layers.Dropout(0.3)(x)
    # Residual branch
    res = layers.Dense(1024)(inp)
    x = layers.Add()([x, res])
    x = layers.Activation('relu')(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(1, activation='linear')(x)
    return tf.keras.Model(inp, out)

mlp_model = build_model_residual_wide(X.shape[1])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()


In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)


## Self‑Norm MLP

In [ ]:
# 3. Self‑Norm MLP, Self‑Normalizing Network with SELU & AlphaDropout
#    Input → 512 → SELU → AD(0.1) → add(input→dense(512))
#          → 512 → SELU → AD(0.1) → add(input→dense(512))       
#          → 256 → SELU → AD(0.1) → add(input→dense(256))
#          → Output


mlp_model = tf.keras.Sequential([
        layers.Dense(512, activation='selu', input_shape=(X.shape[1],)),
        layers.AlphaDropout(0.1),
        layers.Dense(512, activation='selu'),
        layers.AlphaDropout(0.1),
        layers.Dense(256, activation='selu'),
        layers.AlphaDropout(0.1),
        layers.Dense(1, activation='linear')
    ])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # linea ideale y=x
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()


In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)


## Stacked Narrow MLP

In [ ]:
# 4. Stacked Narrow MLP, Deep Narrow‑and‑Deep MLP
#    Input → 256 → LeakyReLU → DO(0.25) → add(input→dense(256))
#          → 128 → LeakyReLU → DO(0.25) → add(input→dense(128))
#          → 128 → LeakyReLU → DO(0.2)  → add(input→dense(128))
#          → 64  → LeakyReLU → DO(0.2)  → add(input→dense(64))
#          → 32  → LeakyReLU → DO(0.2)  → add(input→dense(32))
#          → 16  → LeakyReLU → DO(0.15) → add(input→dense(16))
#          → Output


mlp_model = tf.keras.Sequential([
        layers.Dense(256, input_shape=(X.shape[1],)),
        LeakyReLU(),
        layers.Dropout(0.25),
        layers.Dense(128),
        LeakyReLU(),
        layers.Dropout(0.25),
        layers.Dense(128),
        LeakyReLU(),
        layers.Dropout(0.2),
        layers.Dense(64),
        LeakyReLU(),
        layers.Dropout(0.2),
        layers.Dense(32),
        LeakyReLU(),
        layers.Dropout(0.2),
        layers.Dense(16),
        LeakyReLU(),
        layers.Dropout(0.15),
        layers.Dense(1, activation='linear')
    ])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--') 
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()


In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)


## NoisyWide MLP

In [ ]:
# 5. NoisyWide MLP, Gaussian Noise‑Augmented Wide MLP

mlp_model = tf.keras.Sequential([
        layers.GaussianNoise(0.1, input_shape=(X.shape[1],)),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(1, activation='linear')
    ])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # linea ideale y=x
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()


In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)


## ParallelBranch MLP

In [ ]:
# 6. ParallelBranch MLP, Multi‑Branch Ensemble MLP
# Input
# ├─ Branch A: Dense(512)→ReLU→Dropout(0.3)→Dense(256)→ReLU
# └─ Branch B: Dense(512)→ReLU→Dropout(0.3)→Dense(256)→ReLU
# Concat → Dense(128)→ReLU→Dropout(0.2) → Output

def build_model_multibranch(input_dim):
    inp = layers.Input(shape=(input_dim,))
    # Branch A
    a = layers.Dense(512, activation='relu')(inp)
    a = layers.Dropout(0.3)(a)
    a = layers.Dense(256, activation='relu')(a)
    # Branch B
    b = layers.Dense(512, activation='relu')(inp)
    b = layers.Dropout(0.3)(b)
    b = layers.Dense(256, activation='relu')(b)
    # Merge
    x = layers.Concatenate()([a, b])
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(1, activation='linear')(x)
    return tf.keras.Model(inp, out)

mlp_model = build_model_multibranch(X.shape[1])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()


In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)


## L2Light MLP

In [ ]:
# 7. L2Light MLP, Lightweight L2‑Heavy Regularized MLP

mlp_model = tf.keras.Sequential([
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X.shape[1],)),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        layers.Dropout(0.3),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.005)),
        layers.Dropout(0.2),
        layers.Dense(1, activation='linear')
    ])


# Compile the model
mlp_model.compile(optimizer='adam',
                  loss='mae',  
                  metrics=['mae', 'mse', smape_f])

# Train the model
history = mlp_model.fit(X_train_scaled, y_train,
                         validation_data=(X_val_scaled, y_val),
                         epochs=500, batch_size=16)

epochs = range(1, len(history.history['loss']) + 1)

plt.figure(figsize=(12, 5))

# Plot of the Loss 
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], 'b', label='Training Loss')
plt.plot(epochs, history.history['val_loss'], 'r', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot of MAE 
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['mae'], 'b', label='Training MAE')
plt.plot(epochs, history.history['val_mae'], 'r', label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.title('Training and Validation MAE')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
y_pred = mlp_model.predict(X_test_scaled).flatten()  

# Main metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / (y_test + 1e-8))) * 100  # 1e-8 to avoid division by zero
smape = 100/len(y_test) * np.sum(2 * np.abs(y_pred - y_test) / (np.abs(y_test) + np.abs(y_pred) + 1e-8))

# Print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"MAPE: {mape:.2f}%")
print(f"SMAPE: {smape:.2f}%")


results = pd.DataFrame({
    'ttp_true': y_test,       # real values
    'ttp_pred': y_pred        # predicted values
})

print(results.tail(10))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')  # linea ideale y=x
plt.xlabel("True Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs True Values")
plt.show()


In [ ]:
explainer = shap.Explainer(mlp_model, X_train_scaled_df)  

shap_values = explainer(X_test_scaled_df)

print("SHAP summary plot:")
shap.plots.beeswarm(
    shap_values,
    max_display=len(binary_features + numeric_features)
)
